In [1]:
import pandas as pd
import numpy as np

In [ ]:
def binomial (n, p, size):
    """
    Generate random numbers following a binomial distribution.

    Parameters:
    - n:    Number of trials (number of times an event is repeated).
    - p:    Probability of success for each trial.
    - size: Number of times to repeat the binomial experiment.

    Returns:
    - An array of random numbers representing the number of successes in each experiment.
    """
    return np.random.binomial(n, p, size)

def randint (low, high, size):
    """
    Generate random integers from a discrete uniform distribution.

    Parameters:
    - low:  The minimum (inclusive) value of the range.
    - high: The maximum (exclusive) value of the range.
    - size: The number of random integers to generate.

    Returns:
    - An array of random integers within the specified range.
    """
    return np.random.randint (low, high, size)

# QUESTION 1
There are n products to be produced $\newline$
There are m suppliers which provide the parts to produce the products $\newline$
A product i needs $a_{ij}$ parts ($a_{ij >= 0}$, $i = 1\rightarrow n$, $j=1\rightarrow m $) $\newline$
The preorder cost for part $j$ is $b_j$ $\newline$ 
The demand for the products is random vector $D = (D_1,..., D_n)$ $\newline$
Cost additionaly $l_i$ to satisfy a unit of demand for product $i$, and unit selling price of that unit is $q_i$ $\newline$
The number of parts ordered is $x_j$, the number of units produced is $z_i$, the numbers of remaining parts unused is $y_j$


### First stage

In [ ]:
n   = 8           # number of products
m   = 5           # number of parts to be ordered
S   = 2           # number of scenarios
p_s = [0.5, 0.5]  # probability of each scenario / density

D = np.array([binomial (10, 0.5, n),
              binomial (10, 0.5, n)]) # demand for each product in each scenario matrix [n x S]

b = randint(5, 10, m) # preorder cost for each part

A = []
for i in range(n):
    A.append(np.array(randint(1,10,m)))

A = np.array(A) # bill of materials matrix [m x n]

s = randint (1, 10, m)       # salvage values
l = randint (100, 200, n)    # additional costs
q = randint (1000, 1400, n)  # unit selling prices

c = l - q #cost coefficients